In [70]:
import easyocr
import os.path
import numpy as np
import cv2 
from matplotlib import pyplot as plt
from collections import Counter

In [71]:
reader = easyocr.Reader(['pt'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [79]:
def detection_matrix(list : list) -> list:
    # list -> wordsearch list
    # organizes the easyOCR response to be a matrix with each element being (box, text, confidence)
    matrix = []
    for j, element in enumerate(list):
        cord, detc, acr = element
        letters = [letter.upper() for letter in detc if letter != ' ']
        line = []
        rigth = cord[1][0]
        left = cord[0][0]
        upper = cord[0][1]
        lower_begin = cord[3][1]
        lower_end = cord[2][1]
        width = round((rigth - left) / len(letters))
        height_variation = round((lower_end - lower_begin) / len(letters)) 
        for i in range(len(letters)):
            left_letter = left + (width * i)
            rigth_letter = left + (width * (i + 1))
            upper_letter = upper + (height_variation * i)
            lower_letter = lower_begin + (height_variation * i)
            cord_letter = [left_letter, upper_letter, rigth_letter, lower_letter]  # (x1, y1, x2, y2)
            line.append((cord_letter, letters[i], acr))
        matrix.append(line)
    return matrix

def search_words_matrix(wordsearch_list: list, img) -> list:
    # do the detection in each letter and organizes to be a matrix with each element being (box, text, confidence)
    detection_m = detection_matrix(wordsearch_list)
    matrix = []
    for line in detection_m:
        line_list = []

        #maybe using first detactin e secundo detection letter to be more acure e usem the best acurrece
        for coord, _, _ in line:
            x1_letter, y1_letter, x2_letter, y2_letter = coord

            #letter img
            img_letter = img[y1_letter:y2_letter, x1_letter:x2_letter]
            data_letter =  reader.readtext(img_letter)
            if len(data_letter) == 1:     
                line_list.append(data_letter[0])

        matrix.append(line_list)
    return matrix

def wordsearch_area(wordsearch_list: list, img) -> tuple:
    # uses de detction to return the coordinates of the full wordshearch
    # tuple -> (x1,y1,x2,y2)
    img_h, img_w = img.shape[:2]  
    left_min, top_min, bottom_max, right_max = img_w, img_h, 0, 0
    #left_min = posicao mais a esquerda possivel
    #top_min = posicao mais alto possivel
    for line in wordsearch_list:
        x1 = line[0][0][0]
        y1 = line[0][0][1]
        x2 = line[0][1][0]
        y2 = line[0][2][1]
            
        left_min = min(left_min, x1)
        top_min = min(top_min, y1)
        right_max = max(right_max, x2)
        bottom_max = max(bottom_max, y2)

    return left_min, top_min, right_max, bottom_max

def search_direction(matrix, r : int, c : int, word : str, direction: str):
    # return -> index of last letter (role, colum)
    # r -> role
    # c -> column 
    # dr -> direction role
    # dc -> direction column
    match direction:
        case 'search_ritgh':
            dr = 0
            dc = 1
        case 'search_left':
            dr = 0
            dc = -1
        case 'search_down':
            dr = 1
            dc = 0
        case 'search_up':
            dr = -1
            dc = 0
        case 'search_up_rigth':
            dr = -1
            dc = 1
        case 'search_up_left':
            dr = -1
            dc = -1
        case 'search_down_rigth':
            dr = 1
            dc = 1
        case 'search_down_left':
            dr = 1
            dc = -1
    try:
        for i, letter in enumerate(word):
            if (matrix[r+(i * dr)][c + (i* dc)]).lower() == letter.lower():
                if i == len(word) - 1:
                    last_index = (r+(i * dr), c + (i* dc)) 
                continue
            else:
                return None 
        return last_index
    except (IndexError):
        return None

def words_matrix_position(matrix : list, words_list : list) -> list:
    # find the words in words_list contained in the matrix of letters (wordsearch)
    # and return a list of the possitions of the letters

    directions = ['search_ritgh', 'search_left', 'search_down', 'search_up', 'search_up_rigth', 'search_up_left', 'search_down_rigth', 'search_down_left']
    position_pairs = []  #list of tuples (r1, c1, r2, c2)
    for r in range(len(matrix)):
        for c in range(len(matrix[r])):
                for word in words_list:
                    # checks if the words starts whit the matrix letter
                    if matrix[r][c].lower() == word[0].lower():
                        # try all the directions until one of them works, break if works and try other word
                        for direction in directions:
                            last_index = search_direction(matrix, r, c, word, direction)
                            if last_index is not None:
                                position_pairs.append((r, c , last_index[0], last_index[1]))
                                # if the word was found, remove it
                                words_list.remove(word)
                                break 
    return position_pairs

In [80]:
img_path = '.\data\wordsearch.png'
if os.path.exists(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = reader.readtext(img)
    words_list = [word for  _, word, _ in result if " " not in word]
    wordsearch_list = [res for res in result if " " in res[1]]
    
    
else:
    print(f'{img_path} path dosent exist')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\pedro\AppData\Local\Temp\ipykernel_39912\2485239479.py:1: SyntaxWarning: invalid escape sequence '\d'
  img_path = '.\data\wordsearch.png'
d:\pedro\Programacao\.Projetos\Python\ocr-wordsearch-solver-EasyOcr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [81]:
test = detection_matrix(wordsearch_list)
for line in test:
    for letter in line:
        print(letter)

([np.int32(140), np.int32(36), np.int32(231), np.int32(110)], '0', np.float64(0.4482848259959764))
([np.int32(231), np.int32(36), np.int32(322), np.int32(110)], 'N', np.float64(0.4482848259959764))
([np.int32(322), np.int32(36), np.int32(413), np.int32(110)], 'A', np.float64(0.4482848259959764))
([np.int32(413), np.int32(36), np.int32(504), np.int32(110)], '0', np.float64(0.4482848259959764))
([np.int32(504), np.int32(36), np.int32(595), np.int32(110)], 'B', np.float64(0.4482848259959764))
([np.int32(595), np.int32(36), np.int32(686), np.int32(110)], 'V', np.float64(0.4482848259959764))
([np.int32(686), np.int32(36), np.int32(777), np.int32(110)], 'L', np.float64(0.4482848259959764))
([np.int32(777), np.int32(36), np.int32(868), np.int32(110)], 'E', np.float64(0.4482848259959764))
([np.int32(868), np.int32(36), np.int32(959), np.int32(110)], 'E', np.float64(0.4482848259959764))
([np.int32(959), np.int32(36), np.int32(1050), np.int32(110)], 'P', np.float64(0.4482848259959764))
([np.int3

In [74]:
matrix = search_words_matrix(wordsearch_list, img)

In [75]:
for line in matrix:
    line_lettes = []
    for letter in line:
        line_lettes.append(letter[1])
    print(line_lettes)

['0', 'N', 'A', '0', 'B', 'L', 'E', 'E', 'P', 'R', 'T', 'A', 'B', 'I', 'L', 'I', 'D', 'A', 'D', 'E']
['H', 'H', 'S', 'P', 'L', 'M', 'A', 'F', 'T', 'N', 'E', 'M', 'I', 'D', 'N', 'E', 'S', '1']
['P', 'I', 'N', '0', '0', 'L', 'T', 'D', 'A', 'A', 'E', 'N', 'A', 'H', 'C', 'A', 'U', 'T', 'R', 'I']
['P', 'E', 'R', 'N', 'Q', 'S', 'N', 'A', 'G', 'L', '0', 'S', 'E', 'A', 'R', 'I', 'U', 'T', 'T', 'S', 'N']
['B', 'R', 'I', 'U', 'U', 'C', 'E', 'S', 'A', 'E', '5', 'N', 'R', 'H', 'I', 'F', 'D', 'N', '5', 'A', 'E']
['R', 'B', 'S', 'P', 'E', 'T', 'R', 'N', 'A', 'R', 'A', 'E', 'U', 'P', 'E', 'E', 'A']
['P', 'M', 'E', 'L', 'I', 'R', 'A', 'Ô', 'T', 'T', 'N', 'S', 'G', 'R', 'A', 'A', 'Õ', 'E', 'S']
['R', 'A', 'L', 'M', '0', 'R', 'P', 'G', 'L', 'A', 'N', 'N', 'W', 'E', 'I', 'M', 'R', '5']
['I', 'R', 'R', 'H', 'S', 'S', 'I', 'T', 'L', 'A', 'D', 'A', 'S', 'C', 'C', 'C', 'T', 'T', 'A', 'E']
['0', 'A', 'A', 'C', 'P', 'R', 'O', 'T', 'E', 'Ç', 'Â', '0', 'C', 'R', 'A', 'D', 'I', 'N', 'I', 'F']
['T', 'B', 'E', 'S',